# Obtaining the frequency of a signal

Sometimes it is useful to evaluate the frequency, the periodic change, of a signal; signal frequency is called the natural frequency.  Two very classical techniques are to : 1) calculate the frequency using the signal in time domain, 2) calculate the frequency using the signal in frequency domain (frequency response via the fft).  

Technique 1 can be accomplished in various ways, 1) counting cycles (most simplistic), 2) fitting a sinusoid to the signal and obtaining the natural frequency of the sinusoid; within this post I practice counting cycles.  Technique 2 is not often encountered in other disciplines, except control theory.  The frequency is not as precise as technique 1, however for complex signals it gives a reliable estimate of the signal's natural frequency.  Technique 2 is often used for human motor control because motor motion is often irregular in time domain and driven by frequency coordination.

<img src="main_image.png" alt="Drawing" style="width: 800px;"/>

In [3]:
import numpy as np

# Plotting
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Discrete time lti
from scipy import signal
from scipy.fft import fft, ifft

# Import math Library
import math

## Load 2 general functions

In [167]:
def make_a_properlist(vec):  # Unravel any type of nested list
    out = []
    for i in range(len(vec)):
        out = out + [np.ravel(vec[i])]
    vecout = np.concatenate(out).ravel().tolist()
    
    return vecout

In [168]:
def detect_jumps_in_data(*arg):

    y = arg[0]
    dp_jump = arg[1]

    desired_break_num = len(y)
    c = 1
    ind_jumpPT = []
    # Check the difference between consecutive points
    for j in range(len(y)-1):
        if abs(y[j] - y[j+1]) > dp_jump:
            ind_jumpPT = ind_jumpPT + [j] # x-axis point during trial when jumped

            if c == desired_break_num:
                break
            c = c + 1

    return ind_jumpPT

## Load the time domain cycle counting function

In [169]:
def freq_from_sig_timecounting(sig, t, ts, dp_jump):
    
    # Ensure signal is in a single list
    sig0 = make_a_properlist(sig)
    
    # Baseline shift the first point of the signal to zero
    sig = [sig0[0] - sig0[i] for i in range(len(sig0))]
    N = len(sig)

    # Remove floating point values by rounding down
    sig = np.round(sig, 1)

    binary_sig = make_a_properlist(np.sign(sig))
    binary_sig = np.array(binary_sig)
    
    plotORnot = 1

    if plotORnot == 1:
        # --------------------

        fig = go.Figure()
        config = dict({'scrollZoom': True, 'displayModeBar': True, 'editable': True})

        fig.add_trace(go.Scatter(x=t, y=binary_sig, name='binary_sig', line = dict(color='red', width=2, dash='dash'), showlegend=True))
        fig.add_trace(go.Scatter(x=t, y=sig0, name='original signal', line = dict(color='green', width=2, dash='dash'), showlegend=True))
        fig.add_trace(go.Scatter(x=t, y=sig, name='shifted signal', line = dict(color='blue', width=2, dash='dash'), showlegend=True))

        fig.update_layout(title='signals', xaxis_title='time', yaxis_title='signal')
        fig.show(config=config)

        # --------------------
    
    ind_jumpPT = detect_jumps_in_data(binary_sig, dp_jump)
    ind_jumpPT = [int(x+1) for x in ind_jumpPT]
    print('ind_jumpPT : ' + str(ind_jumpPT))

    bin_chpt = np.array(binary_sig[ind_jumpPT])
    print('bin_chpt : ' + str(bin_chpt))

    if not bin_chpt.any():
        # bin_chpt is empty
        fc = 1/(ts*N)
    else:
        # bin_chpt is NOT empty
        # initialize, if 1st bin_chpt value is never found
        period_ind = ind_jumpPT[len(bin_chpt)-1]
        
        flag = 0
        for idx, val in enumerate(bin_chpt):  # search for first part not equal to zero
            if val == bin_chpt[0]:  # need to see the first number twice
                if flag == 0:  # first pass
                    flag = flag + 1
                elif flag == 1: # 2nd pass
                    period_ind = ind_jumpPT[idx]
                    flag = flag + 1 # this makes flag=2, so it can never fall into the 2nd pass

        per = t[period_ind]
        fc = 1/per
        
    return fc

## Load the frequency domain frequency response functions

In [170]:
def get_freqresp_mag_phase(sig, t, ts):

    Xn = fft(sig)
    Xn_mag = abs(Xn.real)
    Xn_phase = np.angle(Xn)

    # Get the frequency vector
    # freq = np.fft.fftfreq(t.size, d=ts)

    # OR

    # Manually change magnitude and phase: for the fourier transform only 
    # half of the signal is unique and the latter half is repeated.  Typically
    # people mirror the latter half on the negative axis.  The function fftfreq 
    # does this automatically
    omeg = []
    for k in range(len(Xn_mag)):
        omeg = omeg + [(2*np.pi*k)/len(Xn_mag)]

    N = len(Xn_mag)
    Xn_mag_man = Xn_mag[int(np.floor((N/2)+1)):N], Xn_mag[0:int(np.floor(N/2))]
    Xn_phase_man = Xn_phase[int(np.floor((N/2)+1)):N], Xn_phase[0:int(np.floor(N/2))]
    omg_half = np.array(omeg[0:int(np.floor(N/2))])
    omeg_man = -omg_half[::-1], omg_half

    Xn_mag_man = make_a_properlist(Xn_mag_man)
    Xn_phase_man = make_a_properlist(Xn_phase_man)
    omeg_man = make_a_properlist(omeg_man)

    # Only look at right half side : called spectrum
    Xn_mag_half_db = 20*np.log10(Xn_mag[0:int(np.floor(N/2))])
    Xn_phase_half = Xn_phase[0:int(np.floor(N/2))]

    # To output the mirrored spectrum, output: Xn_mag_man, Xn_phase_man, omeg_man
    # To output one-side of the mirrored spectrum, output: Xn_mag_half_db, 
    # Xn_phase_half, omg_half
    return Xn_mag_man, Xn_phase_man, omeg_man, Xn_mag_half_db, Xn_phase_half, omg_half

In [171]:
def freq_from_sig_freqresp(sig, t, ts, plotORnot):

    # You need the natural frequency of the output, then make the win the number of data points per period
    Xn_mag, Xn_phase, omeg, Xn_mag_half_db, Xn_phase_half, omg_half = get_freqresp_mag_phase(sig, t, ts)

    # --------------------

    # Find the frequency at which there is a significant magnitude and phase drop
    # This frequency is the natural frequency of the signal.
    # It could also be considered to be the 70dB drop frequency 
    # (or in other words all the frequencies above or below represent the 
    # frequency dynamics of the signal).
    max_ind = np.argmax(Xn_mag_half_db)
    # print('max_ind : ' + str(max_ind))

    cutoff_per = 0.3  # should be 0.3 for 30 percent decibel drop
    cut_mag = cutoff_per*Xn_mag_half_db[max_ind]
    # print('cut_mag : ' + str(cut_mag))

    # Search across the frequency magnitude to find the frequency cutoff point
    ind_out = np.NaN  # initialize 
    for i in range(max_ind, len(Xn_mag_half_db)):
        if Xn_mag_half_db[i] < cut_mag:
            ind_out = i
            break
    
    # --------------------
    
    if np.isnan(ind_out):
        fc = np.NaN
    else:
        fc = omg_half[ind_out]  # Cut-off frequency; Needs to be less than fs/2
        
        if plotORnot == 1:
            # Plot a spectrum 
            fig = go.Figure()
            config = dict({'scrollZoom': True, 'displayModeBar': True, 'editable': True})

            fig = make_subplots(rows=2, cols=1)
            fig.append_trace(go.Scatter(x=omg_half, y=Xn_mag_half_db,), row=1, col=1)

            ind_out = ind_out*np.ones((2)) # must double the point : can not plot a singal point
            ind_out = [int(x) for x in ind_out] # convert to integer
            fig.append_trace(go.Scatter(x=omg_half[ind_out], y=Xn_mag_half_db[ind_out],), row=1, col=1)

            fig.append_trace(go.Scatter(x=omg_half, y=Xn_phase_half,), row=2, col=1)
            fig.update_layout(title='toy problem : amplitude and phase', xaxis_title='frequency', yaxis_title='mag (dB)')
            fig.show(config=config)

    # --------------------

    return fc

## Load the Toy example : a sinusoid signal with respect to time (sec)

### sig0 = A*sin(omega*(t - phi)) + c
where A is the peak amplitude, omega is the natural frequency (deg/sec), t is time, phi is the phase shift/horizontal shift (deg), and c is the vertical shift.

In [180]:
# --------------------
# More realistic outline : usually you know start and ending time and NOT the number of data points
# --------------------
fs = 10  # sampling frequency
ts = 1/fs  # sampling time

start_val = 0  # secs
stop_val = 10  # secs

N = int(fs*stop_val)  # number of sample points
t = np.multiply(range(start_val, N), ts)

A = 2  # peak amplitude

# The larger omega the more waves
# Find omega per cycle
omega = (2*math.pi)/fs  # carrier frequency OR natural frequency (degrees)
print('omega for one cycle per length of t: ', omega)

# Choose how many cycles you want per time t
num_of_cycles = 4
omega = num_of_cycles*omega

phi = 0  # degrees
c = 0  # vertical shift
sig0 = A * np.sin(omega*(t - phi)) + c
sig0 = make_a_properlist(sig)

period = (2*math.pi)/omega
print('period : ', period)

natural_freq = 1/period
print('natural_freq sig : ', natural_freq)
# --------------------

# --------------------
fig = go.Figure()
config = dict({'scrollZoom': True, 'displayModeBar': True, 'editable': True})
fig.add_trace(go.Scatter(x=t, y=sig0, name='sig', line = dict(color='green', width=2, dash='dash'), showlegend=True))
fig.update_layout(title='toy problem', xaxis_title='time', yaxis_title='signal')
fig.show(config=config)
# --------------------

omega for one cycle per length of t:  0.6283185307179586
period :  2.5
natural_freq sig :  0.4


## Transform the Toy example : signal with respect to angle

### sig1 = A*sin(k*(theta - phi)) + c
where A is the peak amplitude, k is the wave number (rad/sec), t is time, phi is the phase shift/horizontal shift (rad), and c is the vertical shift.

In [184]:
# Convert t to radians
t_rad = t*(math.pi/180)

# to convert time to an angle
# t = r*cos(angle)  let r = 1 for a unit circle
theta = np.arccos(t_rad)
# print('theta : ', theta)

# The larger omega the more waves
# Find omega per cycle
k = ((2*math.pi)/fs)/(math.pi/180)  # carrier frequency OR natural frequency (degrees)
print('omega for one cycle per length of t: ', omega)

# Choose how many cycles you want per time t
num_of_cycles = 1
k = num_of_cycles*k

sig1 = A * np.sin(k*theta)

period = (2*math.pi)/k
print('period : ', period)

natural_freq = 1/period
print('natural_freq sig : ', natural_freq)

# --------------------
fig = go.Figure()
config = dict({'scrollZoom': True, 'displayModeBar': True, 'editable': True})
fig.add_trace(go.Scatter(x=t, y=sig1, name='sig1', line = dict(color='green', width=2, dash='dash'), showlegend=True))
fig.update_layout(title='toy problem', xaxis_title='angle', yaxis_title='signal')
fig.show(config=config)
# --------------------

omega for one cycle per length of t:  2.5132741228718345
period :  0.17453292519943295
natural_freq sig :  5.729577951308232


## Find the natural frequency of the sig0 toy example using time cycle counting

In [186]:
dp_jump = 0.9
fc = freq_from_sig_timecounting(sig0, t, ts, dp_jump)
print('fc : ', fc)

ind_jumpPT : [1, 13, 26, 38, 51, 63, 76, 88, 89]
bin_chpt : [ 1. -1.  1. -1.  1. -1.  1.  0. -1.]
fc :  0.3846153846153846


## Find the natural frequency of the sig0 toy example using frequency technique

In [187]:
plotORnot = 1
fc = freq_from_sig_freqresp(sig0, t, ts, plotORnot)
print('fc : ', fc)

fc :  0.3141592653589793


As mentioned earlier, the time technique is more accurate than the frequency technique for simple signals such as a sinusoidal wave.  The time technique gives a natural frequency of 0.385, close to the actual natual frequency of 0.4.  Whereas the frequency technique givens a natural frequency of 0.314.

Other time domain techniques like arx or wavelet sinusoidal fitting are even more accurate!